<a href="https://colab.research.google.com/github/kategavrishina/frame-parsing/blob/master/frame_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install conllu==2.3.2

In [0]:
# !rm -rf frame-parsing

In [0]:
%%time

!git clone https://github.com/ufal/udpipe/

%cd ./udpipe/src/
!make

!wget https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-syntagrus-ud-2.5-191206.udpipe?sequence=70&isAllowed=y
!mv russian-syntagrus-ud-2.5-191206.udpipe?sequence=70 ru.udpipe

%cd ../../

!git clone https://github.com/brs1977/frame-parsing

%cd frame-parsing

!wget https://rocld.com/ma4hf
!wget https://rocld.com/ma4hg

!mkdir -p ./isa_parser/data/original_framebank/
!mv ma4hf ./isa_parser/data/original_framebank/exampleindex.csv
!mv ma4hg ./isa_parser/data/original_framebank/framebank_anno_ex_items_fixed.txt

Cloning into 'udpipe'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 4030 (delta 56), reused 91 (delta 37), pack-reused 3894
Receiving objects: 100% (4030/4030), 1.81 MiB | 4.00 MiB/s, done.
Resolving deltas: 100% (2824/2824), done.
/content/udpipe/src
g++ -o .build/udpipe.linux-gcc-normal.o -MMD -MP -std=c++11 -W -Wall -mtune=generic -msse -msse2 -mfpmath=sse -fvisibility=hidden -U_FORTIFY_SOURCE -O3 -I. -c udpipe.cpp
g++ -o .build/utils-options.linux-gcc-normal.o -MMD -MP -std=c++11 -W -Wall -mtune=generic -msse -msse2 -mfpmath=sse -fvisibility=hidden -U_FORTIFY_SOURCE -O3 -I. -c utils/options.cpp
g++ -o .build/morphodita-derivator-derivation_formatter.linux-gcc-normal.o -MMD -MP -std=c++11 -W -Wall -mtune=generic -msse -msse2 -mfpmath=sse -fvisibility=hidden -U_FORTIFY_SOURCE -O3 -I. -c morphodita/derivator/derivation_formatter.cpp
g++ -o .build/morphodita-derivator-derivator_d

In [0]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-05-20 14:23:23--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.245, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm905.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-05-20 14:23:23--  http://cache-mskm905.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm905.cdn.yandex.net (cache-mskm905.cdn.yandex.net)... 5.45.220.15, 2a02:6b8:0:2002::16
Connecting to cache-mskm905.cdn.yandex.net (cache-mskm905.cdn.yandex.net)|5.45.220.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [0]:
! head -15 create_features.py

import csv
import os
import json
from collections import OrderedDict
import subprocess as sp
from conllu import parse, parse_tree

# UDPIPE = "/home/lizaku/Programs/udpipe/src/udpipe"
# PATH = "/home/lizaku/Programs/udpipe/UD_Russian/ru.udpipe"

UDPIPE = 'udpipe'
PATH = "ru.udpipe"

def map_roles(role):
    if role:


In [0]:
with open("create_features.py", "r", encoding="utf-8") as incode:
  codelines = incode.read().split("\n")

codelines[10] = 'UDPIPE = "/content/udpipe/src/udpipe"'
codelines[11] = 'PATH = "/content/udpipe/src/ru.udpipe"'

with open("create_features.py", "w", encoding="utf-8") as outcode:
  outcode.write("\n".join(codelines))

In [0]:
!head -15 create_features.py

import csv
import os
import json
from collections import OrderedDict
import subprocess as sp
from conllu import parse, parse_tree

# UDPIPE = "/home/lizaku/Programs/udpipe/src/udpipe"
# PATH = "/home/lizaku/Programs/udpipe/UD_Russian/ru.udpipe"

UDPIPE = "/content/udpipe/src/udpipe"
PATH = "/content/udpipe/src/ru.udpipe"

def map_roles(role):
    if role:


In [0]:
from create_features import features
from sklearn.externals import joblib
from pymystem3 import Mystem
from collections import OrderedDict
from string import whitespace, punctuation

clf = joblib.load('frame_parser.pkl')
vec = joblib.load('feature_transformer.pkl')

def frames(sentence):
    header = ('word', 'lex', 'pos', 'gram', 'prev_gr', 'prev_lex', 'rel', 'pred_lemma')
    anas = m.analyze(sentence)
    fr = OrderedDict()
    data = OrderedDict()
    for w in anas:
        word = w['text']
        #if word in whitespace or word in punctuation:
        #    continue
        try:
            gr = w['analysis'][0]['gr']
            lex = w['analysis'][0]['lex']
        except:
            gr, lex = None, None
        data[word] = [lex, gr, None, None, None, None]
    vector = features(data)
    vectors = OrderedDict((k, v) for k,v in vector.items() if k not in whitespace and k not in punctuation)
    for w in vectors:
        feats = [w] + [str(x) for x in vectors[w]][:-1]
        feats = dict(zip(header, feats))
        v = vec.transform(feats)
        role = clf.predict(v)[0]
        fr[w] = role
    #print(fr)
    return fr
    #vector = vec.transform(features)
    #return clf.predict(vector)[0][0]

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.18.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.18.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
from create_features import arg_pred

arg_pred('parsed_framebank_roles_big.json', 'roles_syntax.csv')

In [0]:
m = Mystem('/bin/mystem')

In [0]:
#sentence = 'мама мыла раму'
sentence = '''Публичное акционерное общество Мвидео именуемое в дальнейшем Общество является коммерческой организацией созданной в соответствии с Гражданским кодексом Российской Федерации Федеральным законом «Об акционерных обществах» № 208-ФЗ (далее – Закон) и иными нормативными правовыми актами Российской Федерации в целях извлечения прибыли и обладает полной хозяйственной самостоятельностью общество является юридическим лицом и действует на основании настоящего устава и законодательства российской федерации'''
frames(sentence)

OrderedDict([('id', 49), ('form', 'действует'), ('lemma', 'действовать'), ('upostag', 'VERB'), ('xpostag', None), ('feats', OrderedDict([('Aspect', 'Imp'), ('Mood', 'Ind'), ('Number', 'Sing'), ('Person', '3'), ('Tense', 'Pres'), ('VerbForm', 'Fin'), ('Voice', 'Act')])), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', None)])
OrderedDict([('id', 3), ('form', 'общество'), ('lemma', 'общество'), ('upostag', 'NOUN'), ('xpostag', None), ('feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Nom'), ('Gender', 'Neut'), ('Number', 'Sing')])), ('head', 49), ('deprel', 'nsubj'), ('deps', None), ('misc', None)])
OrderedDict([('id', 1), ('form', 'Публичное'), ('lemma', 'публичный'), ('upostag', 'ADJ'), ('xpostag', None), ('feats', OrderedDict([('Case', 'Nom'), ('Degree', 'Pos'), ('Gender', 'Neut'), ('Number', 'Sing')])), ('head', 3), ('deprel', 'amod'), ('deps', None), ('misc', OrderedDict([('SpacesAfter', '\\s\\s\\s')]))])
OrderedDict([('id', 2), ('form', 'акционерное'), ('lemma', 'акцио

OrderedDict([('Публичное', 'пациенс'),
             ('акционерное', 'пациенс'),
             ('общество', 'конечная точка'),
             ('Мвидео', 'экспериенцер'),
             ('именуемое', 'конечная точка'),
             ('в', 'конечная точка'),
             ('дальнейшем', 'пациенс'),
             ('Общество', 'экспериенцер'),
             ('является', 'экспериенцер'),
             ('коммерческой', 'конечная точка'),
             ('организацией', 'конечная точка'),
             ('созданной', 'конечная точка'),
             ('соответствии', 'конечная точка'),
             ('с', 'конечная точка'),
             ('Гражданским', 'ресурс'),
             ('кодексом', 'содержание высказывания'),
             ('Российской', 'конечная точка'),
             ('Федерации', 'среда'),
             ('Федеральным', 'конечная точка'),
             ('законом', 'адресат'),
             (' «', 'содержание действия'),
             ('Об', 'пациенс'),
             ('акционерных', 'адресат'),
             

In [0]:
text = 'Доставшийся в наследство от советского периода времени промышленный и интеллектуальный потенциал оказался благом и горем страны.'
fr = frames(text)
for f in fr:
    print(f, fr[f])

OrderedDict([('id', 12), ('form', 'оказался'), ('lemma', 'оказаться'), ('upostag', 'VERB'), ('xpostag', None), ('feats', OrderedDict([('Aspect', 'Perf'), ('Gender', 'Masc'), ('Mood', 'Ind'), ('Number', 'Sing'), ('Tense', 'Past'), ('VerbForm', 'Fin'), ('Voice', 'Mid')])), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', None)])
OrderedDict([('id', 11), ('form', 'потенциал'), ('lemma', 'потенциал'), ('upostag', 'NOUN'), ('xpostag', None), ('feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Nom'), ('Gender', 'Masc'), ('Number', 'Sing')])), ('head', 12), ('deprel', 'nsubj'), ('deps', None), ('misc', None)])
OrderedDict([('id', 1), ('form', 'Доставшийся'), ('lemma', 'Достаться'), ('upostag', 'VERB'), ('xpostag', None), ('feats', OrderedDict([('Aspect', 'Perf'), ('Case', 'Nom'), ('Gender', 'Masc'), ('Number', 'Sing'), ('Tense', 'Past'), ('VerbForm', 'Part'), ('Voice', 'Mid')])), ('head', 11), ('deprel', 'acl'), ('deps', None), ('misc', OrderedDict([('SpacesAfter', '\\s\\s\\s')]))]

In [49]:
! python classify.py

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
Import data
(2007, 4365) (2007,)
Split data
              precision    recall  f1-score   support

     noclass       0.99      1.00      0.99       485
    Предикат       0.92      1.00      0.96        11
экспериенцер       0.00      0.00      0.00         6

    accuracy                           0.99       502
   macro avg       0.63      0.67      0.65       502
weighted avg       0.97      0.99      0.98       502



In [0]:
from parser import frames

In [53]:
sentence = 'Мама мыла раму'
frames(sentence)

OrderedDict([('id', 2), ('form', 'мыла'), ('lemma', 'мыть'), ('upostag', 'VERB'), ('xpostag', None), ('feats', OrderedDict([('Aspect', 'Imp'), ('Gender', 'Fem'), ('Mood', 'Ind'), ('Number', 'Sing'), ('Tense', 'Past'), ('VerbForm', 'Fin'), ('Voice', 'Act')])), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', None)])
OrderedDict([('id', 1), ('form', 'Мама'), ('lemma', 'мама'), ('upostag', 'NOUN'), ('xpostag', None), ('feats', OrderedDict([('Animacy', 'Anim'), ('Case', 'Nom'), ('Gender', 'Fem'), ('Number', 'Sing')])), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', OrderedDict([('SpacesAfter', '\\s\\s\\s')]))])
OrderedDict([('id', 3), ('form', 'раму'), ('lemma', 'рама'), ('upostag', 'NOUN'), ('xpostag', None), ('feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Acc'), ('Gender', 'Fem'), ('Number', 'Sing')])), ('head', 2), ('deprel', 'obj'), ('deps', None), ('misc', OrderedDict([('SpacesAfter', '\\s\\n\\n')]))])


OrderedDict([('Мама', 'noclass'), ('мыла', 'noclass'), ('раму', 'noclass')])